In [17]:
%matplotlib widget

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, FloatLogSlider
from IPython.display import display
import MulensModel as mm
import VBMicrolensing
import matplotlib.lines as mlines

# Fixed binary parameters (circumbinary host)
s2_fixed = 0.2
q2_fixed = 0.925

# Plot limits and styling
X_MIN, X_MAX = -2, 2
Y_MIN, Y_MAX = -2, 2


def plot_circumbinary_caustics(s3, q3 , psi_deg, s2 = s2_fixed, q2 = q2_fixed ):
    psi = np.deg2rad(psi_deg)

    # Masses normalized so that total binary mass = 1
    m1 = 1.0 / (1.0 + q2)
    m2 = q2 * m1
    m3 = q3 * m1

    # Lens positions (binary along x, planet offset by s3 at angle psi)
    z1x = -q2 * s2 / (1.0 + q2)
    z2x = s2 / (1.0 + q2)
    z3x = s3 * np.cos(psi) #-q2_fixed * s2_fixed / (1.0 + q2_fixed) + 
    z3y = s3 * np.sin(psi)

    # Configure VBMicrolensing for triple-lens caustics
    VBM = VBMicrolensing.VBMicrolensing()
    VBM.Tol = 1e-4
    VBM.SetMethod(VBM.Method.Multipoly)
    vbm_params = [
        z1x, 0.0, m1,  # lens 1 (x, y, m)
        z2x, 0.0, m2,  # lens 2
        z3x, z3y, m3,  # lens 3 (planet)
    ]
    VBM.SetLensGeometry(vbm_params)
    caustics = VBM.Multicaustics()
    critcurves = VBM.Multicriticalcurves()

    # Planetary caustic (relative to planet position and orientation)
    splanet = np.hypot(z3x, z3y)
    qplanet = m3 / (m1 + m2)
    planet_caustic = mm.Caustics(s=splanet, q=qplanet)
    px, py = planet_caustic.get_caustics()
    px, py = np.array(px), np.array(py)
    planetorient = np.arctan2(z3y, z3x)
    if planetorient < 0:
        planetorient += 2.0 * np.pi
    pxr = px * np.cos(planetorient) - py * np.sin(planetorient)
    pyr = px * np.sin(planetorient) + py * np.cos(planetorient)

    # Binary-only caustic for reference
    binary_caustic = mm.Caustics(s=s2, q=q2)
    bx, by = binary_caustic.get_caustics()

    # Draw figure
    fig = plt.figure(figsize=(5, 5), dpi=150)
    ax = fig.add_subplot(1, 1, 1)
    plt.subplots_adjust(top=0.9, bottom=0.08, right=0.95, left=0.1, hspace=0.1, wspace=0.2)

    ax.set_xlim(X_MIN, X_MAX)
    ax.set_ylim(Y_MIN, Y_MAX)
    x_left, x_right = ax.get_xlim()
    y_low, y_high = ax.get_ylim()
    ax.set_aspect(abs((x_right - x_left) / (y_high - y_low)))

    # Plot lenses
    ax.scatter([z1x], [0.0], marker='o', s=8, color='goldenrod', label='Binary lenses')
    ax.scatter([z2x], [0.0], marker='o', s=8*q2, color='red')
    ax.scatter([z3x], [z3y], marker='o', s=8, color='black', label='Planet lens')

    # Plot caustics
    for cau in caustics:
        ax.plot(cau[0], cau[1], 'k', alpha=1.0, linewidth=0.8)
    ax.scatter(pxr, pyr, marker='.', s=0.05, color='green', alpha=0.6)
    ax.scatter(bx, by, marker='.', s=0.05, color='blue', alpha=0.1)
    #plot critical curves for the CB system
    for cc in critcurves:
        ax.plot(cc[0], cc[1], color='gray', linestyle = '--', alpha=1.0, linewidth=0.8)
    
    #Add labels for critical curves and caustics
    black_line = mlines.Line2D([], [], color='black', linestyle='-', label='Circumbinary caustic')
    gray_line = mlines.Line2D([], [], color='gray', linestyle='--', label='Critical curves')
    green_line = mlines.Line2D([], [], color='green', linestyle='-', label='Planet caustic')
    blue_line = mlines.Line2D([], [], color='blue', linestyle='-', label='Binary caustic')
    
    #Create legend
    handles, labels = ax.get_legend_handles_labels()
    handles.append(black_line)
    labels.append(black_line.get_label())
    handles.append(gray_line)
    labels.append(gray_line.get_label())
    handles.append(green_line)
    labels.append(green_line.get_label())
    handles.append(blue_line)
    labels.append(blue_line.get_label())
    legend = ax.legend(handles, labels, loc="upper left", fontsize=6)
    ax.add_artist(legend)

    ax.set_title(f's3={s3:.3f}, q3={q3:.2e}, psi={psi_deg:.1f}°  |  s2={s2:.2f}, q2={q2:.3f}')
    plt.show()


# Sliders
s2_slider = FloatSlider(value=0.2, min=0.1, max=1, step=0.05, description='s2')
q2_slider = FloatSlider(value=0.925, min=0.1, max=0.99, step=0.01, description='q2')
s3_slider = FloatSlider(value=0.9, min=0.05, max=2.5, step=0.1, description='s3')
q3_slider = FloatLogSlider(value=1e-3, base=10, min=-5, max=-1, step=0.1, description='q3')  # 1e-8 .. 1e-1
psi_slider = FloatSlider(value=60.0, min=0.0, max=360.0, step=1.0, description='psi (deg)')

ui = widgets.VBox([s3_slider, q3_slider, psi_slider, s2_slider, q2_slider])
out = widgets.interactive_output(
    plot_circumbinary_caustics, {"s3": s3_slider, "q3": q3_slider, "psi_deg": psi_slider, "s2": s2_slider, "q2": q2_slider}
)

display(ui, out)


Output()

In [ ]:
import sys
sys.path.append('/Users/murlidhar.4/Documents/Projects/microlens/triplelens-1.0.7/test/')
from utils import plot_critcaus_srcimgs, pltlkv
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec
import TripleLensing
TRIL = TripleLensing.TripleLensing()
import MulensModel as mm
import numpy as np
import seaborn as sns
import os
from time import time

#meausre the time taken to run this function
start = time()
t0, u0, tE, s2, q2, alpha, s3, q3, psi, rs = 2000., -0.01, 30., 0.2, 0.925, 126.*np.pi/180., 0.9, 1.0*1e-5, 60.0*np.pi/180. , 1e-3
salpha = np.sin(alpha)
calpha = np.cos(alpha)
params = [t0, u0, tE, s2, q2, alpha, s3, q3, psi, rs]
psideg = psi*180./np.pi
#set up lens system
m1 = 1./(1. + q2) #Normalized so that total mass of binary=1. VBMicrolensing - coordinates are in units of Einstein radius for a unitary mass lens
m2 = q2 * m1
m3 = q3 * m1
z1x = -q2*s2/(1. + q2)
z2x = s2/(1. + q2)
z3x = -q2*s2/(1. + q2) + s3*np.cos(psi)
z3y = s3*np.sin(psi)
mlens = [m1, m2, m3]
zlens = [z1x,0.,z2x,0.,z3x,z3y]
number = "1"
gamma = (75./2.)/tE
# source position
ts = np.linspace(t0 - gamma*tE, t0 + gamma*tE, int(2.*gamma*tE*1440./15.))
tsdash = np.linspace(t0 - 5.*tE, t0 + 5.*tE, 10000)
tn = (ts - t0) / tE
tndash = (tsdash - t0) / tE
y1s = u0 * salpha + tn * calpha # source positions
y2s = u0 * calpha - tn * salpha
y1straj = u0 * salpha + tndash * calpha #source positions for source trajectory
y2straj = u0 * calpha - tndash * salpha

#parameters controls the accuracy of finite source calculation
secnum = 120 # divide the source bondary into how many parts
basenum = 5 # the number density of sampled dots among each part
quaderr_Tol = 1e-2 # the Quadrupole test tolerance
relerr_Tol = 1e-2 # the relative error tolerance for magnification

#planet orientation (planet at planet position, star at origin)
splanet = np.sqrt(z3x**2 + z3y**2)
qplanet = m3/(m1+m2)
alphamulens = (2.0*np.pi - alpha)*180./np.pi
planetcom = np.array([(m3*z3x)/(m1+m2+m3),(m3*z3y)/(m1+m2+m3)]) #Position of COM of planet and star
planetcomdist = np.sqrt(planetcom[0]**2 + planetcom[1]**2)
u0planet = np.abs(planetcom[0]*salpha + planetcom[1]*calpha - u0)*np.sign(u0) #impact parameter of planet
slopep = z3y/z3x
slopet = -np.tan(alpha)
beta = np.arctan(np.abs((slopep - slopet)/(1. + slopep*slopet)))
print(round(np.abs(u0planet - u0),6),round(planetcomdist*np.sin(beta),6))

if z3y > 0:
    planetorient = np.arctan2(z3y,z3x)
elif z3y == 0:
    if z3x > 0:
        planetorient = 0.
    elif z3x < 0:
        planetorient = np.pi
else:
    planetorient = 2*np.pi + np.arctan2(z3y,z3x)
alphaplanet = alphamulens - planetorient*180./np.pi
print("splanet = %.4f, qplanet = %.7f, planetcomdist = %.5f, u0planet = %.5f, alphamulens = %.1f, planetorient = %.2f, alphaplanet = %.2f, slopep = %.3f"%(splanet, qplanet, planetcomdist, u0planet, alphamulens, planetorient*180./np.pi, alphaplanet, np.arctan(slopep)*180./np.pi))
    
